In [2]:
import pandas as pd
import seaborn as sns
import glob
import os
from utils import util
from matplotlib import pyplot as plt
import plotly.graph_objects as go

In [2]:
def get_mid_feat_map(run_name, epoch, video_name):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    target_dir = root_dir + "/" + run_name + f"/epoch{epoch}"
    mid_feat_list = pd.DataFrame()
    
    csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
    for csv_file in csv_files:
        mid_feat = pd.read_csv(csv_file)
        mid_feat_list = pd.concat([mid_feat_list, mid_feat])
    
    mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
    mid_feat_list = mid_feat_list.reset_index(drop=True)
    
    mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
    
    video_df = mid_feat_list[mid_feat_list['video_name'] == video_name]
    video_df = video_df.iloc[:, :-2]
    video_df = video_df.reset_index(drop=True)
    video_df_pos = video_df[video_df['emo_pred'] == 1]
    video_df_neg = video_df[video_df['emo_pred'] == 0]
    video_df_pos = video_df_pos.iloc[:, 3:]
    video_df_neg = video_df_neg.iloc[:, 3:]
    video_df_pos = video_df_pos.T
    video_df_neg = video_df_neg.T
    
    # normalize each column from 0 to 1
    # video_df_pos = video_df_pos.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    # video_df_neg = video_df_neg.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    
    # make heatmap
    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    sns.heatmap(video_df_pos, cmap='Reds', yticklabels=False, xticklabels=False)
    plt.title(f'pred positive n_samples: {video_df_pos.shape[1]}')
    
    plt.subplot(1, 2, 2)
    sns.heatmap(video_df_neg, cmap='Reds', yticklabels=False, xticklabels=False)
    plt.title(f'pred negative n_samples: {video_df_neg.shape[1]}')
    
    plt.show()
        
    return video_df

In [3]:
def get_mid_feat_boxplot(run_name, epoch, video_name, standardize=False):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    target_dir = root_dir + "/" + run_name + f"/epoch{epoch}"
    mid_feat_list = pd.DataFrame()
    
    csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
    for csv_file in csv_files:
        mid_feat = pd.read_csv(csv_file)
        mid_feat_list = pd.concat([mid_feat_list, mid_feat])
    
    mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
    mid_feat_list = mid_feat_list.reset_index(drop=True)
    
    mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
    
    video_df = mid_feat_list[mid_feat_list['video_name'] == video_name]
    video_df = video_df.iloc[:, :-2]
    video_df = video_df.reset_index(drop=True)
    
    video_df_pos = video_df[video_df['emo_pred'] == 1]
    video_df_neg = video_df[video_df['emo_pred'] == 0]
    video_df_pos = video_df_pos.iloc[:, 3:]
    video_df_neg = video_df_neg.iloc[:, 3:]
    video_df_pos = video_df_pos.T
    video_df_neg = video_df_neg.T
    
    # standardize
    if standardize:
        video_df_pos = video_df_pos.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
        video_df_neg = video_df_neg.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
    
    # boxplot of mean
    fig = go.Figure()
    fig.add_trace(go.Box(y=video_df_pos, name='positive(n_samples: {})'.format(video_df_pos.shape[1])))
    fig.add_trace(go.Box(y=video_df_neg, name='negative(n_samples: {})'.format(video_df_neg.shape[1])))
    fig.update_layout(
        title=f'Boxplot of mid features in {video_name}',
        legend=dict(orientation='h'),
        )
    fig.show()
    
    return video_df_pos, video_df_neg

In [6]:
def compare_mid_feat(run_name_list, epoch, video_name, legend_list=None, standardization=False, type='hist', random_sample=False):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    fig = go.Figure()
    
    for i, run_name in enumerate(run_name_list):
        target_dir = root_dir + "/" + run_name + f"/epoch{epoch[i]}"
        mid_feat_list = pd.DataFrame()
        
        csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
        for csv_file in csv_files:
            mid_feat = pd.read_csv(csv_file)
            mid_feat_list = pd.concat([mid_feat_list, mid_feat])
        
        mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
        mid_feat_list = mid_feat_list.reset_index(drop=True)
        
        mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
        
        video_df = mid_feat_list[mid_feat_list['video_name'] == video_name]
        video_df = video_df.iloc[:, :-2]
        video_df = video_df.reset_index(drop=True)
        
        video_df_pos = video_df[video_df['emo_pred'] == 1]
        video_df_neg = video_df[video_df['emo_pred'] == 0]
        video_df_pos = video_df_pos.iloc[:, 3:]
        video_df_neg = video_df_neg.iloc[:, 3:]
        video_df_pos = video_df_pos.T
        video_df_neg = video_df_neg.T
        
        if random_sample:
            video_df_pos = video_df_pos.sample(n=1, axis=1)
            video_df_neg = video_df_neg.sample(n=1, axis=1)
            print(video_df_pos.shape)
        
        # flatten video_df_pos and video_df_neg
        video_df_pos = video_df_pos.values.flatten()
        video_df_neg = video_df_neg.values.flatten()
        
        # standardize
        if standardization:
            video_df_pos = (video_df_pos - video_df_pos.mean()) / video_df_pos.std()
            video_df_neg = (video_df_neg - video_df_neg.mean()) / video_df_neg.std()
            
        # boxplot of mean
        if type == 'box':
            if legend_list:
                fig.add_trace(go.Box(y=video_df_pos, name=f'{legend_list[i]}_pred-positive(n_samples:{int(video_df_pos.shape[0] / 512)})', boxmean=True, showlegend=False))
                fig.add_trace(go.Box(y=video_df_neg, name=f'{legend_list[i]}_pred-negative(n_samples:{int(video_df_neg.shape[0] / 512)})', boxmean=True, showlegend=False))
            else:   
                fig.add_trace(go.Box(y=video_df_pos, name=f'{run_name}_pred-positive(n_samples:{int(video_df_pos.shape[0] / 512)})', boxmean=True, showlegend=False))
                fig.add_trace(go.Box(y=video_df_neg, name=f'{run_name}_pred-negative(n_samples:{int(video_df_neg.shape[0] / 512)})', boxmean=True, showlegend=False))
        
        # histogram of mean
        elif type == 'hist':
            if legend_list:
                fig.add_trace(go.Histogram(x=video_df_pos, name=f'{legend_list[i]}_pred-positive(n_samples:{int(video_df_pos.shape[0] / 512)})'))
                fig.add_trace(go.Histogram(x=video_df_neg, name=f'{legend_list[i]}_pred-negative(n_samples:{int(video_df_neg.shape[0] / 512)})'))
            else:   
                fig.add_trace(go.Histogram(x=video_df_pos, name=f'{run_name}_pred-positive(n_samples:{int(video_df_pos.shape[0] / 512)})'))
                fig.add_trace(go.Histogram(x=video_df_neg, name=f'{run_name}_pred-negative(n_samples:{int(video_df_neg.shape[0] / 512)})'))
    
    if type == 'box':
        # fig.update_yaxes(range=[-2, 4])
        # x_label is not displayed
        fig.update_layout(
            title=f'Boxplot of mid features in {video_name}',
            margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation='h', font=dict(size=15))
        )
    elif type == 'hist':
        fig.update_traces(opacity=0.4, histnorm='probability', autobinx=False, xbins=dict(start=-2, end=4, size=0.1))
        # fig.update_xaxes(range=[-2, 2])     
        fig.update_layout(
            barmode='overlay',
            title=f'Histgram of mid features in {video_name}',
            margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation='h', font=dict(size=15))
        )
    fig.show()

In [14]:
def compare_same_sample_mid_feat(run_name_list, epoch, video_name, frame_num, legend_list=None, standardization=False, type='hist'):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    w_size = 30
    fig = go.Figure()
    
    for i, run_name in enumerate(run_name_list):
        target_dir = root_dir + "/" + run_name + f"/epoch{epoch[i]}"
        mid_feat_list = pd.DataFrame()
        
        csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
        for csv_file in csv_files:
            mid_feat = pd.read_csv(csv_file)
            mid_feat_list = pd.concat([mid_feat_list, mid_feat])
        
        mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
        mid_feat_list = mid_feat_list.reset_index(drop=True)
        
        mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
        
        video_df = mid_feat_list[(mid_feat_list['video_name'] == video_name) & (mid_feat_list['frame_id'] == frame_num)]
        if video_df.shape[0] == 0:
            print(f'frame_num: {frame_num} is not found in {run_name}')
            
            return None
        
        video_df = video_df.iloc[:, :-2]
        video_df = video_df.reset_index(drop=True)
        pred = video_df['emo_pred'].values[0]
        video_df = video_df.iloc[:, 3:]
        video_df = video_df.T
        
        # standardize
        if standardization:
            video_df = video_df.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
            
        video_df = video_df.values.flatten()
            
        # boxplot of mean
        if type == 'box':
            if legend_list:
                fig.add_trace(go.Box(y=video_df, name=f'{legend_list[i]}_pred{pred}', boxmean=True, showlegend=False))
            else:   
                fig.add_trace(go.Box(y=video_df, name=f'{run_name}_pred{pred}', boxmean=True, showlegend=False))
        
        # histogram of mean
        elif type == 'hist':
            if legend_list:
                fig.add_trace(go.Histogram(x=video_df, name=f'{legend_list[i]}_pred{pred}'))
            else:   
                fig.add_trace(go.Histogram(x=video_df, name=f'{run_name}_pred{pred}'))
    
    if type == 'box':
        # fig.update_yaxes(range=[-2, 4])
        fig.update_layout(
            title=f'Boxplot of mid features in {video_name} ({frame_num} - {frame_num+w_size})',
            margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation='h', font=dict(size=15))
        )
    elif type == 'hist':
        fig.update_traces(opacity=0.4, histnorm='probability', autobinx=False, xbins=dict(start=-2, end=4, size=0.1))
        # fig.update_xaxes(range=[-2, 2])     
        fig.update_layout(
            barmode='overlay',
            title=f'Histgram of mid features in {video_name}',
            margin=dict(l=0, r=0, t=40, b=0),
            legend=dict(orientation='h', font=dict(size=15))
        )
    fig.show()    

In [46]:
# import tsne
from sklearn.manifold import TSNE

def get_tsne(run_name_list, epoch, video_name, legend_list=None):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    w_size = 30
    fig = go.Figure()
    all_df = pd.DataFrame()
    
    for i, run_name in enumerate(run_name_list):
        target_dir = root_dir + "/" + run_name + f"/epoch{epoch[i]}"
        mid_feat_list = pd.DataFrame()
        
        csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
        for csv_file in csv_files:
            mid_feat = pd.read_csv(csv_file)
            mid_feat_list = pd.concat([mid_feat_list, mid_feat])
        
        mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
        mid_feat_list = mid_feat_list.reset_index(drop=True)
        
        mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
        
        video_df = mid_feat_list[(mid_feat_list['video_name'] == video_name)]
        
        video_df = video_df.iloc[:, :-2]
        video_df = video_df.reset_index(drop=True)
        video_df_pos = video_df[video_df['emo_pred'] == 1]
        video_df_neg = video_df[video_df['emo_pred'] == 0]
        video_df_pos = video_df_pos.iloc[:, 3:]
        video_df_neg = video_df_neg.iloc[:, 3:]
        video_df_pos['label'] = [2*i] * video_df_pos.shape[0]
        video_df_neg['label'] = [2*i+1] * video_df_neg.shape[0]
        
        if i == 0:
            all_df = pd.concat([video_df_pos, video_df_neg])
        else:
            all_df = pd.concat([all_df, video_df_pos, video_df_neg])
        
    # tsne with all_df
    tsne = TSNE(n_components=2, random_state=0)
    tsne_df = tsne.fit_transform(all_df.iloc[:, :-1])
    tsne_df = pd.DataFrame(tsne_df)
    tsne_df['label'] = all_df['label'].values
    
    # plot tsne
    for i in range(len(run_name_list)*2):
        tsne_df_tmp = tsne_df[tsne_df['label'] == i]
        fig.add_trace(go.Scatter(x=tsne_df_tmp[0], y=tsne_df_tmp[1], mode='markers', name=legend_list[i]))
    
    fig.update_traces(opacity=0.5, marker=dict(size=15))   
    fig.update_layout(
        title=f't-SNE of mid features in {video_name}',
        margin=dict(l=0, r=0, t=40, b=0),
        legend=dict(orientation='h', font=dict(size=15))
    )
    fig.show()
        

In [42]:
def get_tsne2(run_name, epoch, video_name_list, legend_list=None, only_pos=False):
    root_dir = "/mnt/iot-qnap3/mochida/medical-care/emotionestimation/reports/PIMD_A"
    w_size = 30
    fig = go.Figure()
    all_df = pd.DataFrame()
    
    for i, video_name in enumerate(video_name_list):
        target_dir = root_dir + "/" + run_name + f"/epoch{epoch}"
        mid_feat_list = pd.DataFrame()
        
        csv_files = glob.glob(os.path.join(target_dir, "**/", "*feat.csv"), recursive=True) 
        for csv_file in csv_files:
            mid_feat = pd.read_csv(csv_file)
            mid_feat_list = pd.concat([mid_feat_list, mid_feat])
        
        mid_feat_list = mid_feat_list.sort_values(by=['img_path'])
        mid_feat_list = mid_feat_list.reset_index(drop=True)
        
        mid_feat_list['video_name'], mid_feat_list['frame_id'] = zip(*mid_feat_list['img_path'].map(util.get_video_name_and_frame_num))
        
        video_df = mid_feat_list[(mid_feat_list['video_name'] == video_name)]
        
        video_df = video_df.iloc[:, :-2]
        video_df = video_df.reset_index(drop=True)
        video_df_pos = video_df[video_df['emo_pred'] == 1]
        video_df_neg = video_df[video_df['emo_pred'] == 0]
        video_df_pos = video_df_pos.iloc[:, 3:]
        video_df_neg = video_df_neg.iloc[:, 3:]
        video_df_pos['label'] = [2*i] * video_df_pos.shape[0]
        video_df_neg['label'] = [2*i+1] * video_df_neg.shape[0]
        
        if i == 0:
            if only_pos:
                all_df = video_df_pos
            else:
                all_df = pd.concat([video_df_pos, video_df_neg])
        else:
            if only_pos:
                all_df = pd.concat([all_df, video_df_pos])
            else:
                all_df = pd.concat([all_df, video_df_pos, video_df_neg])
        
    # tsne with all_df
    tsne = TSNE(n_components=2, random_state=0)
    tsne_df = tsne.fit_transform(all_df.iloc[:, :-1])
    tsne_df = pd.DataFrame(tsne_df)
    tsne_df['label'] = all_df['label'].values
    
    # plot tsne
 
    for i in range(len(video_name_list)*2):
        tsne_df_tmp = tsne_df[tsne_df['label'] == i]
        if only_pos and i % 2 == 0:
            fig.add_trace(go.Scatter(x=tsne_df_tmp[0], y=tsne_df_tmp[1], mode='markers', name=legend_list[i//2]))
            continue
        elif only_pos and i % 2 == 1:
            continue
        
        fig.add_trace(go.Scatter(x=tsne_df_tmp[0], y=tsne_df_tmp[1], mode='markers', name=legend_list[i]))
    
    fig.update_traces(opacity=0.5, marker=dict(size=15))   
    fig.update_layout(
        title=f't-SNE of mid features in {run_name}',
        margin=dict(l=0, r=0, t=40, b=0),
        legend=dict(orientation='h', font=dict(size=15))
    )
    fig.show()

In [24]:
# compare same sample
video_name = 'video21'
frame_num = 1100
compare_same_sample_mid_feat(["4_d_a", "4_d_g", "4_d_h"], [10, 10, 5], video_name, frame_num, ["AU", "Gaze", "HP"], type='box')

In [7]:
# compare each video
video_name = 'video18'
compare_mid_feat(["4_d_a", "4_d_g", "4_d_h"], [10, 10, 5], video_name, ["AU", "Gaze", "HP"], type='box')

In [54]:
get_tsne(["4_d_a", "4_d_g", "4_d_h"], [10, 10, 5], 'video24', ["AU-pred1", "AU-pred0", "Gaze-pred1", "Gaze-pred0", "HP-pred1", "HP-pred0"])

In [55]:
get_tsne(["4_d_a", "4_d_g", "4_d_h"], [10, 10, 5], 'video25', ["AU-pred1", "AU-pred0", "Gaze-pred1", "Gaze-pred0", "HP-pred1", "HP-pred0"])

In [43]:
get_tsne2("4_d_a", 10, ['video18', 'video19', 'video21', 'video24', 'video25'], ["video18-pred1", "video19-pred1", "video21-pred1", "video24-pred1", "video25-pred1"], only_pos=True)

In [44]:
get_tsne2("4_d_g", 10, ['video18', 'video19', 'video21', 'video24', 'video25'], ["video18-pred1", "video19-pred1", "video21-pred1", "video24-pred1", "video25-pred1"], only_pos=True)